In [2]:
import os
import json
import pandas as pd


1. All indication link for drug and disease
2. Get the drug-gene, gene-disease DF 
3. Connect them and merged with indication DF
4. Get the drug-gene-disease indication DF as positive training set

all used link:
    "disease_protein",
    "drug_effect",
    "drug_protein",
    "disease_phenotype_positive"
    

# Init the folder

In [3]:
NODE_FILE = "/playpen/jesse/drug_repurpose/PrimeKG/nodes.csv"
df_node = pd.read_csv(NODE_FILE)

unique_node_types = df_node['node_type'].nunique()
print(f"Number of unique node types: {unique_node_types}")

print("Unique node types:")
print(df_node['node_type'].unique())

print("\nCount of each node type:")
print(df_node['node_type'].value_counts())

df_node

Number of unique node types: 10
Unique node types:
['gene/protein' 'drug' 'effect/phenotype' 'disease' 'biological_process'
 'molecular_function' 'cellular_component' 'exposure' 'pathway' 'anatomy']

Count of each node type:
node_type
biological_process    28642
gene/protein          27671
disease               17080
effect/phenotype      15311
anatomy               14035
molecular_function    11169
drug                   7957
cellular_component     4176
pathway                2516
exposure                818
Name: count, dtype: int64


,node_index,node_id,node_type,node_name,node_source
0,0,9796,gene/protein,PHYHIP,NCBI
1,1,7918,gene/protein,GPANK1,NCBI
2,2,8233,gene/protein,ZRSR2,NCBI
3,3,4899,gene/protein,NRF1,NCBI
4,4,5297,gene/protein,PI4KA,NCBI
...,...,...,...,...,...
129370,129370,R-HSA-936837,pathway,Ion transport by P-type ATPases,REACTOME
129371,129371,R-HSA-997272,pathway,Inhibition of voltage gated Ca2+ channels via...,REACTOME
129372,129372,1062,anatomy,anatomical entity,UBERON
129373,129373,468,anatomy,multi-cellular organism,UBERON


In [4]:
source_counts = df_node.groupby(['node_type', 'node_source']).size().reset_index(name='count')
print(source_counts)

             node_type    node_source  count
0              anatomy         UBERON  14035
1   biological_process             GO  28642
2   cellular_component             GO   4176
3              disease          MONDO  15813
4              disease  MONDO_grouped   1267
5                 drug       DrugBank   7957
6     effect/phenotype            HPO  15311
7             exposure            CTD    818
8         gene/protein           NCBI  27671
9   molecular_function             GO  11169
10             pathway       REACTOME   2516


In [5]:
EDGE_FILE = "/playpen/jesse/drug_repurpose/PrimeKG/edges.csv"  
df_edges = pd.read_csv(EDGE_FILE)

unique_relations = df_edges['relation'].nunique()
print(f"Number of unique relation types: {unique_relations}")

print("\nUnique relation types:")
print(df_edges['relation'].unique())

print("\nRelation counts:")
print(df_edges['relation'].value_counts())

Number of unique relation types: 30

Unique relation types:
['protein_protein' 'drug_protein' 'contraindication' 'indication'
 'off-label use' 'drug_drug' 'phenotype_protein' 'phenotype_phenotype'
 'disease_phenotype_negative' 'disease_phenotype_positive'
 'disease_protein' 'disease_disease' 'drug_effect' 'bioprocess_bioprocess'
 'molfunc_molfunc' 'cellcomp_cellcomp' 'molfunc_protein'
 'cellcomp_protein' 'bioprocess_protein' 'exposure_protein'
 'exposure_disease' 'exposure_exposure' 'exposure_bioprocess'
 'exposure_molfunc' 'exposure_cellcomp' 'pathway_pathway'
 'pathway_protein' 'anatomy_anatomy' 'anatomy_protein_present'
 'anatomy_protein_absent']

Relation counts:
relation
anatomy_protein_present       3036406
drug_drug                     2672628
protein_protein                642150
disease_phenotype_positive     300634
bioprocess_protein             289610
cellcomp_protein               166804
disease_protein                160822
molfunc_protein                139060
drug_effect

# Step1: Get all indication 

In [6]:
indication_relation = "indication"

# Load known indications
indication_edges = df_edges[df_edges['relation'] == indication_relation][['x_index', 'y_index']]
print(indication_edges)

         x_index  y_index
346730     16687    33577
346731     16687    36035
346764     20297    33577
346765     20297    36035
346768     16693    33577
...          ...      ...
5776153    84333    14471
5776154    27527    16634
5776155    38622    16634
5776156    28673    16634
5776158    39497    17237

[18776 rows x 2 columns]


In [7]:
dupes = indication_edges.groupby(
    ['x_index', 'y_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

Empty DataFrame
Columns: [x_index, y_index, count]
Index: []


In [8]:
merged = indication_edges.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='x_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'x_name',
    'node_type': 'x_type'
}).drop(columns=['node_index'])

merged = merged.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='y_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'y_name',
    'node_type': 'y_type'
}).drop(columns=['node_index'])

def clarify_roles(row):
    if row['x_type'] == 'drug':
        return pd.Series({
            'drug_index': row['x_index'],
            'drug_name': row['x_name'],
            'disease_index': row['y_index'],
            'disease_name': row['y_name']
        })
    else:
        return pd.Series({
            'drug_index': row['y_index'],
            'drug_name': row['y_name'],
            'disease_index': row['x_index'],
            'disease_name': row['x_name']
        })

indication_edges_ordered = merged.apply(clarify_roles, axis=1)
indication_edges_ordered = indication_edges_ordered[['drug_index', 'disease_index']]
indication_edges_ordered

,drug_index,disease_index
0,16687,33577
1,16687,36035
2,20297,33577
3,20297,36035
4,16693,33577
...,...,...
18771,14471,84333
18772,16634,27527
18773,16634,38622
18774,16634,28673


In [9]:
dupes = indication_edges_ordered.groupby(
    ['drug_index', 'disease_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

      drug_index  disease_index  count
0          14014          33675      2
1          14014          37888      2
2          14014          39773      2
3          14014          83781      2
4          14014          83791      2
...          ...            ...    ...
9383       20651          28518      2
9384       20651          32680      2
9385       20651          35475      2
9386       20652          32483      2
9387       20653          28197      2

[9388 rows x 3 columns]


In [10]:
indication_edges_ordered = indication_edges_ordered.drop_duplicates()
print(indication_edges_ordered)
dupes = indication_edges_ordered.groupby(
    ['drug_index', 'disease_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

      drug_index  disease_index
0          16687          33577
1          16687          36035
2          20297          33577
3          20297          36035
4          16693          33577
...          ...            ...
9383       14471          84333
9384       16634          27527
9385       16634          38622
9386       16634          28673
9387       17237          39497

[9388 rows x 2 columns]
Empty DataFrame
Columns: [drug_index, disease_index, count]
Index: []


In [11]:
indication_edges_ordered.to_csv('./data_analysis/indication_edges.csv', index=False)

# Step2: Get 2 DF for indication (drug-gene gene-disease)

In [12]:
indication_cot = [
    "drug_protein",
    "disease_protein"
]

indication_cot_edges = df_edges[df_edges['relation'].isin(indication_cot)]
print(indication_cot_edges)

                relation display_relation  x_index  y_index
321075      drug_protein          carrier    14012     7183
321076      drug_protein          carrier    14012     8256
321077      drug_protein          carrier    14013     4107
321078      drug_protein          carrier    14014     1424
321079      drug_protein          carrier    14015     1424
...                  ...              ...      ...      ...
6030122  disease_protein  associated with    28149     4152
6030123  disease_protein  associated with    28181       59
6030124  disease_protein  associated with    31190     5826
6030125  disease_protein  associated with    33606    10422
6030126  disease_protein  associated with    30032    33802

[212128 rows x 4 columns]


In [13]:
disease_gene = indication_cot_edges[indication_cot_edges['relation'] == 'disease_protein'][['x_index', 'y_index']]
drug_gene = indication_cot_edges[indication_cot_edges['relation'] == 'drug_protein'][['x_index', 'y_index']]
print(disease_gene.head(5))
print(drug_gene.head(5))

         x_index  y_index
3235582     7097    28313
3235583     2174    28313
3235584     8038    28313
3235585     5925    28313
3235586      238    28313
        x_index  y_index
321075    14012     7183
321076    14012     8256
321077    14013     4107
321078    14014     1424
321079    14015     1424


In [14]:
merged = disease_gene.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='x_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'x_name',
    'node_type': 'x_type'
}).drop(columns=['node_index'])

merged = merged.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='y_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'y_name',
    'node_type': 'y_type'
}).drop(columns=['node_index'])

def clarify_roles(row):
    if row['x_type'] == 'gene/protein':
        return pd.Series({
            'gene_index': row['x_index'],
            'gene_name': row['x_name'],
            'disease_index': row['y_index'],
            'disease_name': row['y_name']
        })
    else:
        return pd.Series({
            'gene_index': row['y_index'],
            'gene_name': row['y_name'],
            'disease_index': row['x_index'],
            'disease_name': row['x_name']
        })

disease_gene_orderd = merged.apply(clarify_roles, axis=1)
disease_gene_orderd = disease_gene_orderd[['gene_index', 'disease_index']]
disease_gene_orderd = disease_gene_orderd.drop_duplicates()

In [15]:

dupes = disease_gene_orderd.groupby(
    ['gene_index', 'disease_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

Empty DataFrame
Columns: [gene_index, disease_index, count]
Index: []


In [16]:
disease_gene_orderd.to_csv('./data_analysis/disease_gene.csv', index=False)

In [17]:
merged = drug_gene.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='x_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'x_name',
    'node_type': 'x_type'
}).drop(columns=['node_index'])

merged = merged.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='y_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'y_name',
    'node_type': 'y_type'
}).drop(columns=['node_index'])

def clarify_roles(row):
    if row['x_type'] == 'gene/protein':
        return pd.Series({
            'gene_index': row['x_index'],
            'gene_name': row['x_name'],
            'drug_index': row['y_index'],
            'drug_name': row['y_name']
        })
    else:
        return pd.Series({
            'gene_index': row['y_index'],
            'gene_name': row['y_name'],
            'drug_index': row['x_index'],
            'drug_name': row['x_name']
        })

drug_gene_ordered = merged.apply(clarify_roles, axis=1)
drug_gene_ordered = drug_gene_ordered[['gene_index', 'drug_index']]
drug_gene_ordered = drug_gene_ordered.drop_duplicates()
drug_gene_ordered

,gene_index,drug_index
0,7183,14012
1,8256,14012
2,4107,14013
3,1424,14014
4,1424,14015
...,...,...
25648,6069,14491
25649,12188,14490
25650,12188,14491
25651,10452,14712


In [18]:
dupes = drug_gene_ordered.groupby(
    ['gene_index', 'drug_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

Empty DataFrame
Columns: [gene_index, drug_index, count]
Index: []


In [19]:
drug_gene_ordered.to_csv('./data_analysis/drug_gene.csv', index=False)

In [20]:
# ---- Inference Path 1: Disease → Protein → Drug ----
all_inferred_disease_drug_protein = pd.merge(disease_gene_orderd, drug_gene_ordered, on='gene_index')
print(all_inferred_disease_drug_protein)
all_inferred_disease_drug_protein = all_inferred_disease_drug_protein.drop_duplicates()
print(all_inferred_disease_drug_protein)

        gene_index  disease_index  drug_index
0             7097          28313       14679
1             7097          28313       14012
2             7097          28313       14680
3             2174          28313       15852
4             2174          28313       14280
...            ...            ...         ...
556892        4152          28149       14499
556893        4152          28149       14050
556894        4152          28149       15752
556895        5826          31190       14490
556896        5826          31190       14491

[556897 rows x 3 columns]
        gene_index  disease_index  drug_index
0             7097          28313       14679
1             7097          28313       14012
2             7097          28313       14680
3             2174          28313       15852
4             2174          28313       14280
...            ...            ...         ...
556892        4152          28149       14499
556893        4152          28149       14050
556894 

In [21]:
all_inferred_disease_drug_protein.to_csv('./data_analysis/drug_gene_disease.csv', index=False)

In [22]:
dupes = all_inferred_disease_drug_protein.groupby(
    ['gene_index', 'disease_index', 'drug_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

Empty DataFrame
Columns: [gene_index, disease_index, drug_index, count]
Index: []


In [23]:
valid_ddprotein_index = pd.merge(all_inferred_disease_drug_protein, indication_edges_ordered, on=['drug_index', 'disease_index'])
print(valid_ddprotein_index)

      gene_index  disease_index  drug_index
0           1659          28313       14140
1           1659          28313       14161
2           1659          28313       16011
3           1659          28313       15098
4           1659          28313       14178
...          ...            ...         ...
4731        8192          33605       15417
4732         361          33605       14641
4733        2329          33605       14807
4734       11665          33625       20290
4735       22105          33677       14578

[4736 rows x 3 columns]


In [24]:
valid_ddprotein_index.to_csv('./data_analysis/valid_ddprotein_index.csv', index=False)

In [25]:
dupes = valid_ddprotein_index.groupby(
    ['disease_index', 'drug_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] >= 1])

      disease_index  drug_index  count
0             27219       14208      1
1             27219       14478      1
2             27249       14208      1
3             27275       14449      1
4             27285       14560      1
...             ...         ...    ...
1926          84305       14207      1
1927          84310       14223      3
1928          84310       14323     11
1929          84326       15018      1
1930          84327       15018      1

[1931 rows x 3 columns]


# Step3: Get Positive Train set for valid drug-disease-gene

In [26]:
print(valid_ddprotein_index)

      gene_index  disease_index  drug_index
0           1659          28313       14140
1           1659          28313       14161
2           1659          28313       16011
3           1659          28313       15098
4           1659          28313       14178
...          ...            ...         ...
4731        8192          33605       15417
4732         361          33605       14641
4733        2329          33605       14807
4734       11665          33625       20290
4735       22105          33677       14578

[4736 rows x 3 columns]


In [27]:
valid_ddprotein = valid_ddprotein_index.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='drug_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'drug_name', 'node_type': 'drug_type'}).drop(columns=['node_index'])

valid_ddprotein = valid_ddprotein.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='disease_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'disease_name', 'node_type': 'disease_type'}).drop(columns=['node_index'])

valid_ddprotein = valid_ddprotein.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='gene_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'gene_name', 'node_type': 'gene_type'}).drop(columns=['node_index'])


valid_ddprotein = valid_ddprotein.drop_duplicates()
print(valid_ddprotein.head(10))

   gene_index  disease_index  drug_index        drug_name drug_type  \
0        1659          28313       14140      Ziprasidone      drug   
1        1659          28313       14161       Olanzapine      drug   
2        1659          28313       16011         Loxapine      drug   
3        1659          28313       15098        Promazine      drug   
4        1659          28313       14178   Chlorpromazine      drug   
5        1659          28313       14952      Haloperidol      drug   
6        1659          28313       14223      Risperidone      drug   
7        1659          28313       14913  Trifluoperazine      drug   
8        1659          28313       14829      Flupentixol      drug   
9        1659          28313       14323     Aripiprazole      drug   

    disease_name disease_type gene_name     gene_type  
0  schizophrenia      disease    ADRA1A  gene/protein  
1  schizophrenia      disease    ADRA1A  gene/protein  
2  schizophrenia      disease    ADRA1A  gene/prot

In [28]:
valid_ddprotein.to_csv("./data_analysis/valid_ddprotein.csv", index=False)

In [29]:
train_positive_protein = valid_ddprotein.sample(n=500, random_state=42).reset_index(drop=True)
train_positive_protein_index = train_positive_protein[['drug_index', 'disease_index', 'gene_index']]


In [30]:
train_positive_protein.to_csv("./data_analysis/train_positive_protein.csv", index=False)

# Step4: Get drug-disease-phenotype as Positive Train set
disease_phenotype_positive, then can be treated by XX drug

In [31]:
phenotype_cot = [
    "disease_phenotype_positive"
]

phenotype_cot_edges = df_edges[df_edges['relation'].isin(phenotype_cot)]
disease_pheno = phenotype_cot_edges[phenotype_cot_edges['relation'] == 'disease_phenotype_positive'][['x_index', 'y_index']]
disease_pheno

,x_index,y_index
3085246,27472,24442
3085247,27158,22784
3085248,27158,84344
3085249,27158,22488
3085250,27158,22272
...,...,...
5949711,33713,25218
5949712,32207,22574
5949713,32207,26287
5949714,33561,22204


In [32]:
merged = disease_pheno.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='x_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'x_name',
    'node_type': 'x_type'
}).drop(columns=['node_index'])

merged = merged.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='y_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'y_name',
    'node_type': 'y_type'
}).drop(columns=['node_index'])

def clarify_roles(row):
    if row['x_type'] == 'effect/phenotype':
        return pd.Series({
            'phenotype_index': row['x_index'],
            'phenotype_name': row['x_name'],
            'disease_index': row['y_index'],
            'disease_name': row['y_name']
        })
    else:
        return pd.Series({
            'phenotype_index': row['y_index'],
            'phenotype_name': row['y_name'],
            'disease_index': row['x_index'],
            'disease_name': row['x_name']
        })

disease_pheno_ordered = merged.apply(clarify_roles, axis=1)
disease_pheno_ordered = disease_pheno_ordered[['phenotype_index', 'disease_index']]
disease_pheno_ordered = disease_pheno_ordered.drop_duplicates()


In [33]:
dupes = disease_pheno_ordered.groupby(
    ['phenotype_index', 'disease_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])
print(disease_pheno_ordered)

Empty DataFrame
Columns: [phenotype_index, disease_index, count]
Index: []
        phenotype_index  disease_index
0                 24442          27472
1                 22784          27158
2                 84344          27158
3                 22488          27158
4                 22272          27158
...                 ...            ...
150331            25218          33713
150332            22574          32207
150333            26287          32207
150334            22204          33561
150335            24588          38100

[150317 rows x 2 columns]


In [34]:
print(indication_edges_ordered)

      drug_index  disease_index
0          16687          33577
1          16687          36035
2          20297          33577
3          20297          36035
4          16693          33577
...          ...            ...
9383       14471          84333
9384       16634          27527
9385       16634          38622
9386       16634          28673
9387       17237          39497

[9388 rows x 2 columns]


In [35]:
valid_ddphenotype_index = pd.merge(indication_edges_ordered, disease_pheno_ordered, on='disease_index', how='inner').drop_duplicates()
valid_ddphenotype_index.to_csv("./data_analysis/valid_ddphenotype_index.csv", index=False)

In [36]:
valid_ddphenotype = valid_ddphenotype_index.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='drug_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'drug_name', 'node_type': 'drug_type'}).drop(columns=['node_index'])

valid_ddphenotype = valid_ddphenotype.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='disease_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'disease_name', 'node_type': 'disease_type'}).drop(columns=['node_index'])

valid_ddphenotype = valid_ddphenotype.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='phenotype_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'phenotype_name', 'node_type': 'phenotype_type'}).drop(columns=['node_index'])


valid_ddphenotype = valid_ddphenotype.drop_duplicates()
print(valid_ddphenotype.head(10))

   drug_index  disease_index  phenotype_index   drug_name drug_type  \
0       16687          33577            94180  Fosinopril      drug   
1       16687          33577            23513  Fosinopril      drug   
2       16687          33577            94390  Fosinopril      drug   
3       16687          33577            22741  Fosinopril      drug   
4       20297          33577            94180   Imidapril      drug   
5       20297          33577            23513   Imidapril      drug   
6       20297          33577            94390   Imidapril      drug   
7       20297          33577            22741   Imidapril      drug   
8       16693          33577            94180  Cilazapril      drug   
9       16693          33577            23513  Cilazapril      drug   

            disease_name disease_type  \
0  hypertensive disorder      disease   
1  hypertensive disorder      disease   
2  hypertensive disorder      disease   
3  hypertensive disorder      disease   
4  hypertensi

In [37]:
valid_ddphenotype.to_csv("./data_analysis/valid_ddphenotype.csv", index=False)

In [38]:
# First, sample 500 for training
train_positive_phenotype = valid_ddphenotype.sample(n=500, random_state=42).reset_index(drop=True)
train_positive_phenotype_index = train_positive_phenotype[['drug_index', 'disease_index', 'phenotype_index']]

In [39]:
train_positive_phenotype.to_csv("./data_analysis/train_positive_phenotype.csv", index=False)

# Step5: Get the remaining indication edges and select 500 as test set

In [40]:
phenotype_pairs = train_positive_phenotype_index[['drug_index', 'disease_index']]
protein_pairs = train_positive_protein_index[['drug_index', 'disease_index']]

merged_train_index = pd.concat([phenotype_pairs, protein_pairs]).drop_duplicates().reset_index(drop=True)
merged_train_index.drop_duplicates()

,drug_index,disease_index
0,15772,28547
1,20597,33643
2,14042,29078
3,15860,32325
4,14390,33609
...,...,...
769,15146,38957
770,14956,38242
771,14310,37703
772,17605,29375


In [41]:
remaining_indication_index = indication_edges_ordered.merge(
    merged_train_index,
    on=['drug_index', 'disease_index'],
    how='left',
    indicator=True
)

remaining_indication_index = remaining_indication_index[remaining_indication_index['_merge'] == 'left_only'].drop(columns=['_merge'])
remaining_indication_index

,drug_index,disease_index
0,16687,33577
1,16687,36035
2,20297,33577
3,20297,36035
4,16693,33577
...,...,...
9383,14471,84333
9384,16634,27527
9385,16634,38622
9386,16634,28673


In [47]:
overlap = remaining_indication_index.merge(merged_train_index, on=['disease_index', 'drug_index'], how='inner')
if not overlap.empty:
    print("There is overlap:")
    print(overlap)
else:
    print("No overlap found.")

No overlap found.


In [42]:
phenotype_edges = df_edges[df_edges['relation'] == 'disease_phenotype_positive']
protein_edges = df_edges[df_edges['relation'] == 'disease_protein']

In [43]:
def get_related_items(disease_id, edges, target='phenotype'):
    matches_x = edges[edges['x_index'] == disease_id]['y_index'].tolist()
    matches_y = edges[edges['y_index'] == disease_id]['x_index'].tolist()
    return list(set(matches_x + matches_y))

remaining_indication_pg = remaining_indication_index
disease_ids = remaining_indication_index['disease_index'].unique()

disease_to_phenotypes = {
    d: get_related_items(d, phenotype_edges, target='phenotype') for d in disease_ids
}
disease_to_proteins = {
    d: get_related_items(d, protein_edges, target='protein') for d in disease_ids
}

remaining_indication_pg['related_phenotypes'] = remaining_indication_pg['disease_index'].map(disease_to_phenotypes)
remaining_indication_pg['related_proteins'] = remaining_indication_pg['disease_index'].map(disease_to_proteins)


filtered_remaining_indication = remaining_indication_pg[
    remaining_indication_pg['related_phenotypes'].apply(lambda x: len(x) > 0) &
    remaining_indication_pg['related_proteins'].apply(lambda x: len(x) > 0)
].reset_index(drop=True)
filtered_remaining_indication

,drug_index,disease_index,related_phenotypes,related_proteins
0,16687,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
1,20297,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
2,16693,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
3,14944,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
4,15762,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
...,...,...,...,...
2549,15858,31384,"[91907, 91908, 24582, 84361, 22282, 23179, 926...","[10376, 4113, 6931, 2967, 13210, 3106, 10021, ..."
2550,15858,30905,"[85504, 22788, 22791, 85005, 22675, 22681, 244...","[10376, 4113, 6931, 2967, 13210, 3106, 10021, ..."
2551,17286,27446,"[22658, 22788, 24199, 23816, 86664, 87179, 848...","[9001, 7684, 12150]"
2552,16634,27527,"[90898, 85019, 90273, 88994, 84774, 84519, 230...","[2515, 11356]"


In [44]:
test_positive = filtered_remaining_indication.sample(n=501, random_state=42).reset_index(drop=True)
test_positive.to_csv("./data_analysis/test_positive_new.csv", index=False)
test_positive

,drug_index,disease_index,related_phenotypes,related_proteins
0,14520,33651,"[94334, 22550]","[1641, 150]"
1,15098,33683,"[94533, 22540, 25518, 94454, 94520]","[3552, 417, 8834, 5667, 4968, 2889, 7083, 1004..."
2,15493,29637,"[25479, 84488, 90495, 25620, 25599, 84764, 225...","[3104, 2401, 2789, 3046, 1884, 22023, 5320, 10..."
3,14275,30654,"[23970, 86308, 22757, 84890]","[2, 9221, 1549, 4111, 13840, 4114, 1558, 1559,..."
4,20375,33714,"[93737, 22987, 22988, 94534]","[2978, 9029, 4425, 12842, 1740, 12766, 4959]"
...,...,...,...,...
496,15179,33703,"[22539, 94372, 94243, 94244]","[12305, 1587, 6229, 1485]"
497,14269,32531,"[22952, 25547, 23214, 24143, 85040, 24337, 256...","[769, 134, 6926, 4369, 7186, 7441, 8212, 1558,..."
498,15451,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
499,14196,28851,"[22791, 87306, 85009, 22933, 22934, 86296, 225...","[2840, 177, 7212, 9965]"


In [45]:
test_positive_with_drug = test_positive.merge(
    df_node,
    left_on='drug_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'drug_name',
    'node_type': 'drug_type'
}).drop(columns=['node_index'])

test_positive_full = test_positive_with_drug.merge(
    df_node,
    left_on='disease_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'disease_name',
    'node_type': 'disease_type'
}).drop(columns=['node_index'])

columns_to_keep = [
    'drug_index',
    'disease_index',
    'drug_type',
    'drug_name',
    'disease_type',
    'disease_name',
    'related_phenotypes',
    'related_proteins'
]

cleaned_test_positive = test_positive_full[columns_to_keep]
cleaned_test_positive

,drug_index,disease_index,drug_type,drug_name,disease_type,disease_name,related_phenotypes,related_proteins
0,14520,33651,drug,Allantoin,disease,seborrheic dermatitis,"[94334, 22550]","[1641, 150]"
1,15098,33683,drug,Promazine,disease,urticaria (disease),"[94533, 22540, 25518, 94454, 94520]","[3552, 417, 8834, 5667, 4968, 2889, 7083, 1004..."
2,15493,29637,drug,Bromocriptine,disease,prolactin producing pituitary gland tumor,"[25479, 84488, 90495, 25620, 25599, 84764, 225...","[3104, 2401, 2789, 3046, 1884, 22023, 5320, 10..."
3,14275,30654,drug,Doxorubicin,disease,acute lymphoblastic/lymphocytic leukemia,"[23970, 86308, 22757, 84890]","[2, 9221, 1549, 4111, 13840, 4114, 1558, 1559,..."
4,20375,33714,drug,Aluminum hydroxide,disease,esophagitis (disease),"[93737, 22987, 22988, 94534]","[2978, 9029, 4425, 12842, 1740, 12766, 4959]"
...,...,...,...,...,...,...,...,...
496,15179,33703,drug,Nicotinamide,disease,acne (disease),"[22539, 94372, 94243, 94244]","[12305, 1587, 6229, 1485]"
497,14269,32531,drug,Methylprednisolone,disease,mantle cell lymphoma,"[22952, 25547, 23214, 24143, 85040, 24337, 256...","[769, 134, 6926, 4369, 7186, 7441, 8212, 1558,..."
498,15451,33577,drug,Bisoprolol,disease,hypertensive disorder,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
499,14196,28851,drug,Propranolol,disease,Kasabach-Merritt syndrome,"[22791, 87306, 85009, 22933, 22934, 86296, 225...","[2840, 177, 7212, 9965]"


In [46]:
cleaned_test_positive.to_csv("./data_analysis/test_positive_full_new.csv", index=False)

# Step6: Get all negative sample from the contraindication edges

In [48]:
contraindication_relation = "contraindication"

contraindication_edges = df_edges[df_edges['relation'] == contraindication_relation][['x_index', 'y_index']]
print(contraindication_edges)

         x_index  y_index
346728     15193    33577
346729     15193    36035
346732     14483    33577
346733     14483    36035
346734     16476    33577
...          ...      ...
5776145    35751    14251
5776146    35846    20456
5776147    35751    20456
5776148    27446    17286
5776157    84334    18277

[61350 rows x 2 columns]


In [49]:
dupes = contraindication_edges.groupby(
    ['x_index', 'y_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

Empty DataFrame
Columns: [x_index, y_index, count]
Index: []


In [50]:
merged = contraindication_edges.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='x_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'x_name',
    'node_type': 'x_type'
}).drop(columns=['node_index'])

merged = merged.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='y_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'y_name',
    'node_type': 'y_type'
}).drop(columns=['node_index'])

def clarify_roles(row):
    if row['x_type'] == 'drug':
        return pd.Series({
            'drug_index': row['x_index'],
            'drug_name': row['x_name'],
            'disease_index': row['y_index'],
            'disease_name': row['y_name']
        })
    else:
        return pd.Series({
            'drug_index': row['y_index'],
            'drug_name': row['y_name'],
            'disease_index': row['x_index'],
            'disease_name': row['x_name']
        })

contraindication_edges_ordered = merged.apply(clarify_roles, axis=1)
contraindication_edges_ordered = contraindication_edges_ordered[['drug_index', 'disease_index']]
contraindication_edges_ordered = contraindication_edges_ordered.drop_duplicates()

In [51]:
print(contraindication_edges_ordered)
dupes = indication_edges_ordered.groupby(
    ['drug_index', 'disease_index']
).size().reset_index(name='count')

# Filter for duplicates
print(dupes[dupes['count'] > 1])

       drug_index  disease_index
0           15193          33577
1           15193          36035
2           14483          33577
3           14483          36035
4           16476          33577
...           ...            ...
30670       14251          35751
30671       20456          35846
30672       20456          35751
30673       17286          27446
30674       18277          84334

[30675 rows x 2 columns]
Empty DataFrame
Columns: [drug_index, disease_index, count]
Index: []


In [52]:
contraindication_edges_ordered.to_csv('./data_analysis/contraindication_edges.csv', index=False)

# Step7: Get all drug disease phenotype for Negative Train set

In [53]:
# contraindication_edges_ordered

contra_ddphenotype_index = pd.merge(contraindication_edges_ordered, disease_pheno_ordered, on='disease_index', how='inner').drop_duplicates()
contra_ddphenotype_index.to_csv("./data_analysis/contra_ddphenotype_index.csv", index=False)
contra_ddphenotype_index

,drug_index,disease_index,phenotype_index
0,15193,33577,94180
1,15193,33577,23513
2,15193,33577,94390
3,15193,33577,22741
4,14483,33577,94180
...,...,...,...
179779,17286,27446,22354
179780,17286,27446,22699
179781,17286,27446,23101
179782,17286,27446,22658


In [54]:
train_negative_phenotype_index = contra_ddphenotype_index.sample(n=500, random_state=42).reset_index(drop=True)
train_negative_phenotype_index

,drug_index,disease_index,phenotype_index
0,17064,27626,84771
1,14249,32719,25974
2,14976,31650,84675
3,15116,33091,22437
4,14561,31502,24199
...,...,...,...
495,16071,28158,23157
496,20303,31122,23368
497,14026,32899,89372
498,14385,32909,24227


In [55]:
train_negative_phenotype = train_negative_phenotype_index.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='drug_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'drug_name', 'node_type': 'drug_type'}).drop(columns=['node_index'])

train_negative_phenotype = train_negative_phenotype.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='disease_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'disease_name', 'node_type': 'disease_type'}).drop(columns=['node_index'])

train_negative_phenotype = train_negative_phenotype.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='phenotype_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'phenotype_name', 'node_type': 'phenotype_type'}).drop(columns=['node_index'])


train_negative_phenotype = train_negative_phenotype.drop_duplicates()
print(train_negative_phenotype.head(10))

   drug_index  disease_index  phenotype_index             drug_name drug_type  \
0       17064          27626            84771           Meprobamate      drug   
1       14249          32719            25974       Pseudoephedrine      drug   
2       14976          31650            84675             Triazolam      drug   
3       15116          33091            22437            Paroxetine      drug   
4       14561          31502            24199       Cholecalciferol      drug   
5       20157          27703            22859           Guaifenesin      drug   
6       15821          28052            23011  Isosorbide dinitrate      drug   
7       14861          32714            25663           Neostigmine      drug   
8       15908          32967            22484         Glutamic acid      drug   
9       14833          31033            94148            Riboflavin      drug   

                                 disease_name disease_type  \
0                            thrombocytopenia 

In [56]:
train_negative_phenotype.to_csv("./data_analysis/train_negative_phenotype.csv", index=False)

# Step8: Get all drug disease gene for Negative Train set

In [57]:
contra_ddgene_index = pd.merge(contraindication_edges_ordered, disease_gene_orderd, on='disease_index', how='inner').drop_duplicates()
contra_ddgene_index.to_csv("./data_analysis/contra_ddgene_index.csv", index=False)
contra_ddgene_index

,drug_index,disease_index,gene_index
0,15193,33577,3912
1,15193,33577,8805
2,15193,33577,2667
3,15193,33577,4794
4,15193,33577,10901
...,...,...,...
1308217,20456,35751,35031
1308218,17286,27446,9001
1308219,17286,27446,7684
1308220,17286,27446,12150


In [58]:
train_negative_gene_index = contra_ddgene_index.sample(n=500, random_state=42).reset_index(drop=True)
train_negative_gene_index

,drug_index,disease_index,gene_index
0,14198,36187,4497
1,15495,37703,1876
2,15078,36035,10901
3,14806,83760,10047
4,14286,27933,2551
...,...,...,...
495,18277,83760,3466
496,15783,35641,1863
497,15776,37703,2591
498,14476,35966,1497


In [59]:
train_negative_gene = train_negative_gene_index.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='drug_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'drug_name', 'node_type': 'drug_type'}).drop(columns=['node_index'])

train_negative_gene = train_negative_gene.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='disease_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'disease_name', 'node_type': 'disease_type'}).drop(columns=['node_index'])

train_negative_gene = train_negative_gene.merge(
    df_node[['node_index', 'node_name', 'node_type']],
    left_on='gene_index',
    right_on='node_index',
    how='left'
).rename(columns={'node_name': 'gene_name', 'node_type': 'gene_type'}).drop(columns=['node_index'])


train_negative_gene = train_negative_gene.drop_duplicates()
print(train_negative_gene.head(10))

   drug_index  disease_index  gene_index               drug_name drug_type  \
0       14198          36187        4497               Clonidine      drug   
1       15495          37703        1876               Estazolam      drug   
2       15078          36035       10901                Tramadol      drug   
3       14806          83760       10047              Entacapone      drug   
4       14286          27933        2551              Rifampicin      drug   
5       14476          36527         554  Testosterone cypionate      drug   
6       14235          32617        2423               Estradiol      drug   
7       14293          28407        8336               Glipizide      drug   
8       14629          27933        8757              Tryptophan      drug   
9       14990          33605        8805        Chlorpheniramine      drug   

                                disease_name disease_type gene_name  \
0                    coronary artery disease      disease      NOS3   

In [60]:
train_negative_gene.to_csv("./data_analysis/train_negative_gene.csv", index=False)

# Step9: Get all remaining contraindication edges, 

In [61]:
gene_neg_pairs = train_negative_gene_index[['drug_index', 'disease_index']]
phenotype_neg_pairs = train_negative_phenotype_index[['drug_index', 'disease_index']]

merged_neg_train_index = pd.concat([phenotype_neg_pairs, gene_neg_pairs]).drop_duplicates().reset_index(drop=True)
merged_neg_train_index

,drug_index,disease_index
0,17064,27626
1,14249,32719
2,14976,31650
3,15116,33091
4,14561,31502
...,...,...
937,18277,83760
938,15783,35641
939,15776,37703
940,14476,35966


In [62]:
remaining_contraindication_index = contraindication_edges_ordered.merge(
    merged_neg_train_index,
    on=['drug_index', 'disease_index'],
    how='left',
    indicator=True
)

remaining_contraindication_index = remaining_contraindication_index[remaining_contraindication_index['_merge'] == 'left_only'].drop(columns=['_merge'])
remaining_contraindication_index

,drug_index,disease_index
0,15193,33577
1,15193,36035
2,14483,33577
3,14483,36035
4,16476,33577
...,...,...
30670,14251,35751
30671,20456,35846
30672,20456,35751
30673,17286,27446


In [70]:
overlap = remaining_contraindication_index.merge(merged_neg_train_index, on=['disease_index', 'drug_index'], how='inner')
if not overlap.empty:
    print("There is overlap:")
    print(overlap)
else:
    print("No overlap found.")

No overlap found.


In [63]:
def get_related_items(disease_id, edges, target='phenotype'):
    matches_x = edges[edges['x_index'] == disease_id]['y_index'].tolist()
    matches_y = edges[edges['y_index'] == disease_id]['x_index'].tolist()
    return list(set(matches_x + matches_y))

remaining_contraindication_pg = remaining_contraindication_index
disease_ids = remaining_contraindication_index['disease_index'].unique()

disease_to_phenotypes = {
    d: get_related_items(d, phenotype_edges, target='phenotype') for d in disease_ids
}
disease_to_proteins = {
    d: get_related_items(d, protein_edges, target='protein') for d in disease_ids
}

remaining_contraindication_pg['related_phenotypes'] = remaining_contraindication_pg['disease_index'].map(disease_to_phenotypes)
remaining_contraindication_pg['related_proteins'] = remaining_contraindication_pg['disease_index'].map(disease_to_proteins)


filtered_remaining_contraindication = remaining_contraindication_pg[
    remaining_contraindication_pg['related_phenotypes'].apply(lambda x: len(x) > 0) &
    remaining_contraindication_pg['related_proteins'].apply(lambda x: len(x) > 0)
].reset_index(drop=True)

filtered_remaining_contraindication

,drug_index,disease_index,related_phenotypes,related_proteins
0,15193,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
1,14483,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
2,16476,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
3,20148,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
4,15087,33577,"[23513, 94180, 22741, 94390]","[8805, 3912, 11080, 2667, 33776, 4497, 22002, ..."
...,...,...,...,...
10538,16741,33579,"[94337, 94338, 22570, 26061, 24371, 94421, 243...","[2002, 2228, 34124]"
10539,16741,27462,"[22569, 22759, 24389, 33725]","[2002, 2228, 34124]"
10540,14919,27761,"[23172, 26375, 85515, 85131, 86929, 89121, 234...","[7665, 1682, 2103]"
10541,16593,28591,"[23168, 25608, 25865, 25596, 24461, 26127, 262...",[1722]


In [75]:
test_negative = filtered_remaining_contraindication.sample(n=502, random_state=42).reset_index(drop=True)
test_negative.to_csv("./data_analysis/test_negative_new.csv", index=False)

In [76]:
test_negative

,drug_index,disease_index,related_phenotypes,related_proteins
0,20391,28811,"[25699, 26180, 22759, 22472, 85642, 85419, 850...","[7457, 12289, 5829, 7302, 13706, 34890, 10413,..."
1,15294,28547,"[25858, 84357, 24327, 25224, 24331, 23564, 226...","[8192, 22017, 519, 11, 8212, 9237, 1567, 6175,..."
2,14783,33679,"[94408, 23033]","[1760, 8805, 12165, 172, 213, 1494, 509]"
3,20239,29188,"[22124, 22933, 26180]","[6671, 6002, 1908, 8733, 3231]"
4,17243,27326,"[89225, 94223, 26257, 84369, 94225, 84887, 238...","[7177, 9233, 34962, 7198, 1968, 177, 10417, 35..."
...,...,...,...,...
497,14168,31493,"[24224, 87330, 26180, 22759, 86573, 23950, 861...","[12165, 33925, 33926, 10376, 33929, 2698, 3405..."
498,20427,31918,"[88450, 85125, 22277, 84616, 22156, 24462, 245...","[9905, 10181]"
499,20350,30491,"[26449, 84853, 25854, 22759]","[5203, 341, 9207]"
500,15776,27933,"[22759, 22447]","[8192, 10257, 22, 6175, 33, 2082, 2096, 6195, ..."


In [77]:
overlap = test_negative.merge(merged_neg_train_index, on=['disease_index', 'drug_index'], how='inner')
if not overlap.empty:
    print("There is overlap:")
    print(overlap)
else:
    print("No overlap found.")

No overlap found.


In [78]:
test_negative_with_drug = test_negative.merge(
    df_node,
    left_on='drug_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'drug_name',
    'node_type': 'drug_type'
}).drop(columns=['node_index'])

test_negative_full = test_negative_with_drug.merge(
    df_node,
    left_on='disease_index',
    right_on='node_index',
    how='left'
).rename(columns={
    'node_name': 'disease_name',
    'node_type': 'disease_type'
}).drop(columns=['node_index'])

columns_to_keep = [
    'drug_index',
    'disease_index',
    'drug_type',
    'drug_name',
    'disease_type',
    'disease_name',
    'related_phenotypes',
    'related_proteins'
]

cleaned_test_negative = test_negative_full[columns_to_keep]
cleaned_test_negative

,drug_index,disease_index,drug_type,drug_name,disease_type,disease_name,related_phenotypes,related_proteins
0,20391,28811,drug,Silver sulfadiazine,disease,cystinuria,"[25699, 26180, 22759, 22472, 85642, 85419, 850...","[7457, 12289, 5829, 7302, 13706, 34890, 10413,..."
1,15294,28547,drug,Tacrine,disease,Parkinson disease,"[25858, 84357, 24327, 25224, 24331, 23564, 226...","[8192, 22017, 519, 11, 8212, 9237, 1567, 6175,..."
2,14783,33679,drug,Caffeine,disease,potassium deficiency disease,"[94408, 23033]","[1760, 8805, 12165, 172, 213, 1494, 509]"
3,20239,29188,drug,Penicillamine,disease,autosomal recessive severe congenital neutrope...,"[22124, 22933, 26180]","[6671, 6002, 1908, 8733, 3231]"
4,17243,27326,drug,Alimemazine,disease,long QT syndrome,"[89225, 94223, 26257, 84369, 94225, 84887, 238...","[7177, 9233, 34962, 7198, 1968, 177, 10417, 35..."
...,...,...,...,...,...,...,...,...
497,14168,31493,drug,Spironolactone,disease,pancreatitis,"[24224, 87330, 26180, 22759, 86573, 23950, 861...","[12165, 33925, 33926, 10376, 33929, 2698, 3405..."
498,20427,31918,drug,Cholestyramine,disease,phenylketonuria,"[88450, 85125, 22277, 84616, 22156, 24462, 245...","[9905, 10181]"
499,20350,30491,drug,Pentaerithrityl,disease,obstructive sleep apnea syndrome,"[26449, 84853, 25854, 22759]","[5203, 341, 9207]"
500,15776,27933,drug,Nabilone,disease,anxiety disorder,"[22759, 22447]","[8192, 10257, 22, 6175, 33, 2082, 2096, 6195, ..."


In [79]:
cleaned_test_negative.to_csv("./data_analysis/test_negative_full_new.csv", index=False)

# Drug to Drug

In [ ]:
import os
root_dir = "/playpen/hongxuan/drug_repurpose/PrimeKG"
NODE_FILE = root_dir + "/nodes.csv"
EDGE_FILE = root_dir + "/edges.csv"
OUTPUT_FOLDER = root_dir + "/outputs/"
OUTPUT_DIR = OUTPUT_FOLDER + "/drug2drug_2hop_strong"

## Drug to Drug (1-hop)

In [21]:


OUTPUT_DIR = OUTPUT_FOLDER + "/drug2drug_1hop"
os.makedirs(OUTPUT_DIR, exist_ok = True)
kg = pd.read_csv(root_dir + "/kg_grouped.csv")
kg = kg.query("relation == 'drug_drug'").iloc[:, [4, 8]]
kg.to_csv(OUTPUT_DIR + "/drug_drug.csv",index = False)

/tmp/ipykernel_349124/2108557556.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  kg = pd.read_csv(root_dir + "/kg_grouped.csv")


## Drug to Drug (2-hop)

提供的关系类型:

['ppi', 'carrier', 'enzyme', 'target', 'transporter', 'contraindication', 'indication', 'off-label use', 'synergistic interaction', 'associated with', 'parent-child', 'phenotype absent', 'phenotype present', 'side effect', 'interacts with', 'linked to', 'expression present', 'expression absent']

**分析不同 2-hop 路径的推断强度 (Drug A -> X -> Drug B):**

1. **`Drug A --[rel1]--> Protein P <--[rel2]-- Drug B`**
    - **强推断信号 (最可能推出新用途):**
        - `rel1 = target`, `rel2 = target`: 两个药都靶向同一个蛋白。这是最强的信号，表明它们可能具有非常相似的核心作用机制。如果 A 能治 Z 病，B 很可能也行（如果蛋白 P 和 Z 病相关）。
        - `rel1 = enzyme`, `rel2 = enzyme`: 两个药都以酶相关的方式作用于同一个蛋白（例如都是抑制剂，或都是底物）。这也表明它们可能影响同一条代谢通路或信号通路，是较强的相似性信号。
        - `rel1 = target`, `rel2 = enzyme` (或反之): 两个药都功能性地作用于同一个蛋白，但方式略有不同。这也是一个较强的信号，表明它们都参与了涉及蛋白 P 的生物过程。
    - **中等推断信号:**
        - `rel1 = transporter`, `rel2 = transporter`: 都使用同一个转运蛋白。可能暗示相似的吸收分布代谢排泄(ADME)特性或作用于表达该转运蛋白的特定细胞/组织。如果转运蛋白本身是治疗靶点或对药物递送至关重要，则意义更强。
        - `rel1 = carrier`, `rel2 = carrier`: 类似 transporter，意义可能稍弱。
    - **弱推断信号 (推断新用途的依据较弱):**
        - `rel1`, `rel2` 是不同类型的功能性互作，如 `target` vs `carrier` (就像 F8 的例子)。它们确实都接触了同一个蛋白，但作用方式和目的可能完全不同，很难据此推断它们有相似的治疗效果。
        - `rel1`, `rel2` 包含 `ppi`, `interacts with`, `associated with`, `linked to` 等比较模糊的关系：这些关系不够具体，推断能力弱。需要看更精确的关系类型。
2. **`Drug A --[rel1]--> Disease D <--[rel2]-- Drug B`**
    - **强推断信号:**
        - `rel1 = indication`, `rel2 = indication`: 两个药都被批准治疗同一种疾病。强烈暗示它们都能有效调节该疾病的生物学。如果 A 还治 Z 病，这为 B 可能也能治 Z 病提供了有力支持。
        - `rel1 = off-label use`, `rel2 = off-label use`: 类似 indication，但基于临床实践。同样是强信号。
        - `rel1 = indication`, `rel2 = off-label use` (或反之): 也是强信号。
    - **弱推断信号 (用于推断 *新* 用途):**
        - `rel1 = contraindication`, `rel2 = contraindication`: 都禁用于该疾病。表明它们可能存在相似的风险或不良作用。这本身不能用来推断药物 B 能获得药物 A 的 *适应症*，但揭示了它们在特定疾病背景下的某种功能共性（负面）。
        - `rel1 = indication`, `rel2 = contraindication` (或反之): 关系冲突，几乎无法用于推断 B 具有 A 的适应症。
    - **极弱推断信号:**
        - `associated with`, `linked to`: 关系太模糊，意义不明。
3. **`Drug A --[rel1]--> Phenotype P <--[rel2]-- Drug B`**
    - **中等/弱推断信号:**
        - `rel1 = side effect`, `rel2 = side effect`: 两个药引起同样的副作用。如前所述，这 *可能* 暗示共同机制或脱靶效应，但常常不具备特异性。如果副作用非常独特和罕见，这个连接的意义会增强。普通副作用（恶心、头痛）意义不大。
        - `rel1 = phenotype present`, `rel2 = phenotype present`: 类似 side effect。
    - **极弱推断信号:**
        - `phenotype absent`: 关系不清晰，难以推断。

**回到你的具体例子:**

- **路径:** `AICC --[enzyme]--> F8 --[carrier]--> Copper` (或者反过来看 `Copper --[carrier]--> F8 <--[enzyme]-- AICC`)
- **关系类型:** `enzyme` 和 `carrier`。
- **推断能力:** 如上分析，这属于 `Drug -> Protein -> Drug` 路径中，`rel1` 和 `rel2` 功能类型不同的情况。它们确实都与 F8 相互作用，但方式不同，功能也不同（一个是凝血调控，一个是微量元素）。因此，**从 AICC 的已知用途（治疗血友病）推断 Copper 也能治疗血友病（或者反过来），依据非常薄弱。** 不能仅仅因为它们都 "碰巧" 与 F8 有关系，就认为它们可以互相重定位。

**关于 `Drug -> Disease -> Drug` 是否可以推断:**

**绝对可以**。正如上面分析的，如果两个药物**都**被用于治疗（`indication` 或 `off-label use`）同一种疾病，这强烈表明它们在功能或机制上存在某种相似性或关联性，是药物重定位中一个非常有价值的推断依据。

**总结:**

在利用 2-hop 路径进行药物重定位时，你必须仔细考察**中间节点 X 的类型**以及**两条边的 `relationship` 类型**。

- **最强的推断依据通常来自:**
    - `Drug -> Protein -> Drug` 路径，且两个关系都是 `target` 或 `enzyme`（表明相似的作用机制）。
    - `Drug -> Disease -> Drug` 路径，且两个关系都是 `indication` 或 `off-label use`（表明相似的临床疗效）。
- **其他路径和关系组合** 也可能提供线索，但证据强度较弱，需要更多旁证或更深入的生物学分析来支持你的重定位假设。

In [13]:
import pandas as pd
import os
import numpy as np
import concurrent.futures
import math


# --- CPU Core Configuration ---
NUM_WORKERS = os.cpu_count()
print(f"Using {NUM_WORKERS} CPU workers for parallel processing.")

if OUTPUT_DIR and not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# --- Load Data ---
print("Loading data...")
try:
    df_node = pd.read_csv(NODE_FILE)
    df_edges = pd.read_csv(EDGE_FILE)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"Error: Cannot find node or edge files at the specified paths:")
    print(f"Node file: {NODE_FILE}")
    print(f"Edge file: {EDGE_FILE}")
    exit()

print(f"Nodes: {len(df_node)}")
print(f"Edges: {len(df_edges)}")

# --- Prepare Node DataFrames ---
print("Preparing node data...")
df_drug = df_node[df_node['node_type'] == 'drug'].copy()
df_protein = df_node[df_node['node_type'] == 'gene/protein'].copy()
df_disease = df_node[df_node['node_type'] == 'disease'].copy()
# df_phenotype is no longer needed

# Helper function to merge node details
def merge_node_info(df, node_df, index_col, prefix):
    """Merges node information onto a DataFrame."""
    node_df_subset = node_df[['node_index', 'node_id', 'node_name', 'node_source']].copy()
    return df.merge(
        node_df_subset,
        left_on=index_col,
        right_on='node_index',
        how='left'
    ).rename(columns={
        'node_id': f'{prefix}_id',
        'node_name': f'{prefix}_name',
        'node_source': f'{prefix}_source'
    }).drop(columns=['node_index'])

# --- Worker Functions for Parallel Role Clarification ---
# (Worker functions clarify_drug_protein_roles_chunk, clarify_drug_disease_roles_chunk 保持不变)
def clarify_drug_protein_roles_chunk(df_chunk):
    processed_rows = []
    for _, row in df_chunk.iterrows():
        if row['x_type'] == 'drug' and row['y_type'] == 'gene/protein':
            processed_rows.append({
                'drug_index': row['x_index'],
                'protein_index': row['y_index'],
                'relation_type': row['display_relation']
            })
        elif row['y_type'] == 'drug' and row['x_type'] == 'gene/protein':
             processed_rows.append({
                'drug_index': row['y_index'],
                'protein_index': row['x_index'],
                'relation_type': row['display_relation']
            })
    return processed_rows

def clarify_drug_disease_roles_chunk(df_chunk):
    processed_rows = []
    for _, row in df_chunk.iterrows():
        if row['x_type'] == 'drug' and row['y_type'] == 'disease':
            processed_rows.append({
                'drug_index': row['x_index'],
                'disease_index': row['y_index'],
                'relation_type': row['relation']
            })
        elif row['y_type'] == 'drug' and row['x_type'] == 'disease':
             processed_rows.append({
                'drug_index': row['y_index'],
                'disease_index': row['x_index'],
                'relation_type': row['relation']
            })
    return processed_rows


# --- 1. Extract STRONG Drug -> Protein -> Drug ---
print("\nExtracting STRONG Drug -> Protein -> Drug paths...")

drug_protein_edges_raw = df_edges[df_edges['relation'] == 'drug_protein'][['x_index', 'y_index', 'display_relation']].copy()
drug_protein_edges_merged = drug_protein_edges_raw.merge(
    df_node[['node_index', 'node_type']].rename(columns={'node_type': 'x_type'}),
    left_on='x_index', right_on='node_index', how='left'
).drop(columns=['node_index']).merge(
    df_node[['node_index', 'node_type']].rename(columns={'node_type': 'y_type'}),
    left_on='y_index', right_on='node_index', how='left'
).drop(columns=['node_index'])

print(f"Clarifying drug-protein roles using {NUM_WORKERS} workers...")
chunks = np.array_split(drug_protein_edges_merged, NUM_WORKERS)
all_processed_rows = []
with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    future_to_chunk = {executor.submit(clarify_drug_protein_roles_chunk, chunk): i for i, chunk in enumerate(chunks)}
    for future in concurrent.futures.as_completed(future_to_chunk):
        try: all_processed_rows.extend(future.result())
        except Exception as exc: print(f'Chunk (protein) generated an exception: {exc}')
drug_protein_edges = pd.DataFrame(all_processed_rows).drop_duplicates()
print("Role clarification done.")

print("Performing merges for Drug-Protein-Drug...")
drug_protein_drug_paths = drug_protein_edges.merge(
    drug_protein_edges, on='protein_index', suffixes=('_1', '_2')
)
drug_protein_drug_paths = drug_protein_drug_paths[drug_protein_drug_paths['drug_index_1'] != drug_protein_drug_paths['drug_index_2']]

# *** FILTERING FOR STRONG SIGNALS (Protein) ***
print("Filtering for strong protein relationships (target/enzyme)...")
strong_protein_rels = {'target', 'enzyme'} # 定义强关联类型集合
strong_drug_protein_drug_paths = drug_protein_drug_paths[
    drug_protein_drug_paths['relation_type_1'].isin(strong_protein_rels) &
    drug_protein_drug_paths['relation_type_2'].isin(strong_protein_rels)
].copy() # 确保是副本以避免警告
print(f"Initial paths: {len(drug_protein_drug_paths)}, Strong paths retained: {len(strong_drug_protein_drug_paths)}")
# *** END FILTERING ***

print("Merging node details for strong protein paths...")
# 在过滤后的数据上合并节点信息
df_temp = merge_node_info(strong_drug_protein_drug_paths, df_drug, 'drug_index_1', 'drug1')
df_temp = merge_node_info(df_temp, df_drug, 'drug_index_2', 'drug2')
strong_drug_protein_drug_df = merge_node_info(df_temp, df_protein, 'protein_index', 'protein')
print("Merges complete.")

# Select and reorder columns
strong_drug_protein_drug_df = strong_drug_protein_drug_df[[
    'drug_index_1', 'drug1_id', 'drug1_name', 'drug1_source', 'relation_type_1',
    'protein_index', 'protein_id', 'protein_name', 'protein_source',
    'drug_index_2', 'drug2_id', 'drug2_name', 'drug2_source', 'relation_type_2'
]].drop_duplicates().reset_index(drop=True)
print(f"Generated {len(strong_drug_protein_drug_df)} STRONG Drug-Protein-Drug paths.")


# --- 2. Extract STRONG Drug -> Disease -> Drug ---
print("\nExtracting STRONG Drug -> Disease -> Drug paths...")
relation_types_disease = ['indication', 'contraindication', 'off-label use'] # 初始需要包含禁忌症以正确澄清角色
drug_disease_edges_raw = df_edges[df_edges['relation'].isin(relation_types_disease)][['x_index', 'y_index', 'relation']].copy()
drug_disease_edges_merged = drug_disease_edges_raw.merge(
    df_node[['node_index', 'node_type']].rename(columns={'node_type': 'x_type'}),
    left_on='x_index', right_on='node_index', how='left'
).drop(columns=['node_index']).merge(
    df_node[['node_index', 'node_type']].rename(columns={'node_type': 'y_type'}),
    left_on='y_index', right_on='node_index', how='left'
).drop(columns=['node_index'])

print(f"Clarifying drug-disease roles using {NUM_WORKERS} workers...")
chunks = np.array_split(drug_disease_edges_merged, NUM_WORKERS)
all_processed_rows = []
with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    future_to_chunk = {executor.submit(clarify_drug_disease_roles_chunk, chunk): i for i, chunk in enumerate(chunks)}
    for future in concurrent.futures.as_completed(future_to_chunk):
            try: all_processed_rows.extend(future.result())
            except Exception as exc: print(f'Chunk (disease) generated an exception: {exc}')
drug_disease_edges = pd.DataFrame(all_processed_rows).drop_duplicates()
print("Role clarification done.")

print("Performing merges for Drug-Disease-Drug...")
drug_disease_drug_paths = drug_disease_edges.merge(
    drug_disease_edges, on='disease_index', suffixes=('_1', '_2')
)
drug_disease_drug_paths = drug_disease_drug_paths[drug_disease_drug_paths['drug_index_1'] != drug_disease_drug_paths['drug_index_2']]

# *** FILTERING FOR STRONG SIGNALS (Disease) ***
print("Filtering for strong disease relationships (indication/off-label use)...")
strong_disease_rels = {'indication', 'off-label use'} # 定义强关联类型集合
strong_drug_disease_drug_paths = drug_disease_drug_paths[
    drug_disease_drug_paths['relation_type_1'].isin(strong_disease_rels) &
    drug_disease_drug_paths['relation_type_2'].isin(strong_disease_rels)
].copy() # 确保是副本
print(f"Initial paths: {len(drug_disease_drug_paths)}, Strong paths retained: {len(strong_drug_disease_drug_paths)}")
# *** END FILTERING ***

print("Merging node details for strong disease paths...")
# 在过滤后的数据上合并节点信息
df_temp = merge_node_info(strong_drug_disease_drug_paths, df_drug, 'drug_index_1', 'drug1')
df_temp = merge_node_info(df_temp, df_drug, 'drug_index_2', 'drug2')
strong_drug_disease_drug_df = merge_node_info(df_temp, df_disease, 'disease_index', 'disease')
print("Merges complete.")

# Select and reorder columns
strong_drug_disease_drug_df = strong_drug_disease_drug_df[[
    'drug_index_1', 'drug1_id', 'drug1_name', 'drug1_source', 'relation_type_1',
    'disease_index', 'disease_id', 'disease_name', 'disease_source',
    'drug_index_2', 'drug2_id', 'drug2_name', 'drug2_source', 'relation_type_2'
]].drop_duplicates().reset_index(drop=True)
print(f"Generated {len(strong_drug_disease_drug_df)} STRONG Drug-Disease-Drug paths.")

# --- Section for Drug -> Phenotype -> Drug removed ---

# --- Display Results & Save (Optional) ---
print("\n--- STRONG Drug -> Protein -> Drug (Sample) ---")
if not strong_drug_protein_drug_df.empty:
    print(strong_drug_protein_drug_df.head())
else:
    print("No strong Drug-Protein-Drug paths found.")

print("\n--- STRONG Drug -> Disease -> Drug (Sample) ---")
if not strong_drug_disease_drug_df.empty:
    print(strong_drug_disease_drug_df.head())
else:
    print("No strong Drug-Disease-Drug paths found.")

if OUTPUT_DIR:
    print(f"\nSaving results to {OUTPUT_DIR}...")
    strong_drug_protein_drug_df.to_csv(os.path.join(OUTPUT_DIR, 'strong_drug_protein_drug_paths.csv'), index=False)
    strong_drug_disease_drug_df.to_csv(os.path.join(OUTPUT_DIR, 'strong_drug_disease_drug_paths.csv'), index=False)
    # Removed saving for phenotype paths
    print("Files saved.")

print("\nExtraction complete.")


Using 255 CPU workers for parallel processing.
Loading data...
Data loaded successfully.
Nodes: 129375
Edges: 8100498
Preparing node data...

Extracting STRONG Drug -> Protein -> Drug paths...
Clarifying drug-protein roles using 255 workers...


/home/swyun/miniconda3/envs/idrs/lib/python3.13/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Role clarification done.
Performing merges for Drug-Protein-Drug...
Filtering for strong protein relationships (target/enzyme)...
Initial paths: 2971352, Strong paths retained: 2289938
Merging node details for strong protein paths...
Merges complete.
Generated 2289938 STRONG Drug-Protein-Drug paths.

Extracting STRONG Drug -> Disease -> Drug paths...
Clarifying drug-disease roles using 255 workers...


/home/swyun/miniconda3/envs/idrs/lib/python3.13/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Role clarification done.
Performing merges for Drug-Disease-Drug...
Filtering for strong disease relationships (indication/off-label use)...
Initial paths: 5219046, Strong paths retained: 256538
Merging node details for strong disease paths...
Merges complete.
Generated 256538 STRONG Drug-Disease-Drug paths.

--- STRONG Drug -> Protein -> Drug (Sample) ---
   drug_index_1 drug1_id drug1_name drug1_source relation_type_1  \
0         15098  DB00420  Promazine     DrugBank          enzyme   
1         15098  DB00420  Promazine     DrugBank          enzyme   
2         15098  DB00420  Promazine     DrugBank          enzyme   
3         15098  DB00420  Promazine     DrugBank          enzyme   
4         15098  DB00420  Promazine     DrugBank          enzyme   

   protein_index protein_id protein_name protein_source  drug_index_2  \
0           3955       1544       CYP1A2           NCBI         15099   
1           3955       1544       CYP1A2           NCBI         15298   
2           3